In [1]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [2]:
data_path = 'data/doc_level-sentiment/doc_level'

## Vocabulary Indexing

### Note: 

Pay attention how you represent your sequences as an input of RNN model.
With a fixed length vector, you will need to pad the shorter sequences with "0".
Consequently, your vocabulary indexing needs to consider this "0" as padding.

### Function to create vocabulary index

### Returns:

Python dictionary format of vocabulary indexing

In [3]:
num_regex = re.compile('^[+-]?[0-9]+\.?[0-9]*$')

def create_vocab(domain, data_path, maxlen=0, vocab_size=0):
    
    print('Creating vocab ...')

    f = os.path.join(data_path,'%s_text.txt'%(domain))

    total_words, unique_words = 0, 0
    word_freqs = {}

    fin = codecs.open(f, 'r', 'utf-8')
    for line in fin:
        words = line.split()
        if maxlen > 0 and len(words) > maxlen:
            continue

        for w in words:
            if not bool(num_regex.match(w)):
                try:
                    word_freqs[w] += 1
                except KeyError:
                    unique_words += 1
                    word_freqs[w] = 1
                total_words += 1

    print ('  %i total words, %i unique words' % (total_words, unique_words))
    sorted_word_freqs = sorted(word_freqs.items(), key=operator.itemgetter(1), reverse=True)

    vocab = {'<pad>':0, '<unk>':1, '<num>':2}
    index = len(vocab)
    for word, _ in sorted_word_freqs:
        vocab[word] = index
        index += 1
        if vocab_size > 0 and index > vocab_size + 2:
            break
    if vocab_size > 0:
        print (' keep the top %i words' % vocab_size)

  
    return vocab

## Sequence preprocessing

### Function to transform word sequence -> integer sequence

### Note:

The raw data set has 5 class labels. Here, we only consider 3 sentiment classes.

### Returns:

integer sequence of text, corresponding labels (int), maxlen (maximum length of sequences)

In [4]:
def create_data(vocab, text_path, label_path, domain, skip_top, skip_len, replace_non_vocab):
    
    data = []
    label = [] # {pos: 0, neg: 1, neu: 2}
    
    f = codecs.open(text_path, 'r', 'utf-8')
    f_l = codecs.open(label_path, 'r', 'utf-8')
    
    num_hit, unk_hit, skip_top_hit, total = 0., 0., 0., 0.
    pos_count, neg_count, neu_count = 0, 0, 0
    max_len = 0

    for line, score in zip(f, f_l):
        word_indices = []
        words = line.split()
        if skip_len > 0 and len(words) > skip_len:
            continue

        score = float(score.strip())
        if score < 3:
            neg_count += 1
            label.append(1)
        elif score > 3:
            pos_count += 1
            label.append(0)
        else:
            neu_count += 1
            label.append(2)
          
        for word in words:
            if bool(num_regex.match(word)):
                word_indices.append(vocab['<num>'])
                num_hit += 1
            elif word in vocab:
                word_ind = vocab[word]
                if skip_top > 0 and word_ind < skip_top + 3:
                    skip_top_hit += 1
                else:
                    word_indices.append(word_ind)
            else:
                if replace_non_vocab:
                    word_indices.append(vocab['<unk>'])
                unk_hit += 1
            total += 1

        if len(word_indices) > max_len:
            max_len = len(word_indices)

        data.append(word_indices)

    f.close()
    f_l.close()

    print('  <num> hit rate: %.2f%%, <unk> hit rate: %.2f%%' % (100*num_hit/total, 100*unk_hit/total))

    print (domain)
    print( 'pos count: ', pos_count )
    print( 'neg count: ', neg_count )
    print( 'neu count: ', neu_count )

    return np.array(data), np.array(label), max_len


### Main Preprocessing Function

### Call : 

- create_vocab()
- create_data()

### Return

- vocabulary index
- integer sequence (model input)
- label (model output)
- maximum sequence length -> as parameter for RNN / LSTM

In [5]:
def prepare_data(domain, data_path, vocab_size, skip_top=0, skip_len=0, replace_non_vocab=1):
    
    print(domain)

    assert domain in ['amazon_electronics', 'yelp14']

    vocab = create_vocab(domain, data_path, skip_len, vocab_size)

    text_path = os.path.join(data_path,'%s_text.txt'%(domain))
    score_path = os.path.join(data_path,'%s_label.txt'%(domain))

    data, label, max_len = create_data(vocab, text_path, score_path, domain, skip_top, \
                                       skip_len, replace_non_vocab)

    return vocab, data, label, max_len

## Run Preprocessing

In [6]:
# choose domain data to train
domain_name = 'amazon_electronics'

In [7]:
vocab, data_list, label_list, overall_maxlen = prepare_data(domain_name, data_path, 10000)

amazon_electronics
Creating vocab ...
  3440972 total words, 39122 unique words
 keep the top 10000 words
  <num> hit rate: 1.04%, <unk> hit rate: 1.56%
amazon_electronics
pos count:  10000
neg count:  10000
neu count:  10000


In [8]:
overall_maxlen

1016

In [9]:
data_list.shape

(30000,)

In [10]:
print(data_list[0])

[168, 109, 12, 791, 11, 17, 2207, 218, 9, 48, 1, 117, 3, 1318, 9181, 37, 1003, 16, 3, 535, 74, 8, 806, 6, 134, 348, 1610, 60, 523, 522, 6, 540, 10, 35, 149, 8, 458, 3412, 51, 104, 10, 39, 113, 6, 4918, 15, 13, 105, 173, 61, 42, 8, 173, 36, 47, 64, 61, 11, 3, 657, 825, 25, 69, 20, 93, 21, 3, 9181, 79, 760, 3, 236, 2175, 5047, 2258, 14, 42, 6, 3243, 9, 905, 51, 8165, 301, 3, 951, 211, 3, 1432, 10, 35, 3, 1432, 10, 994, 1944, 114, 343, 308, 89, 185, 5, 174, 56, 3, 105, 16, 3, 2520, 10, 8, 3412, 66, 9, 259, 31, 15, 6, 4932, 7, 13, 24, 39, 17, 856, 4, 1469, 3, 1099, 7, 522, 368, 10, 17, 1040, 4, 437, 6, 1040, 101, 16, 630, 11, 48, 1873, 36, 37, 1003, 41, 3, 121, 138, 36]


In [11]:
label_list.shape

(30000,)

In [12]:
print(label_list[0])

0


### Example of how to access the stored vocabulary indexing

In [13]:
print(list(vocab.items())[:5])

[('requiring', 2350), ('reach', 1311), ('defective', 879), ('seattle', 8930), ('debate', 8876)]


In [14]:
vocab['love']

301

In [15]:
idx_words = dict((v,k) for (k,v) in vocab.items())

In [16]:
print(list(idx_words.items())[:5])

[(0, '<pad>'), (1, '<unk>'), (2, '<num>'), (3, 'the'), (4, 'i')]


### Storing all preprocessing data

Here, we store as a pickle format

In [17]:
import _pickle as cPickle

In [18]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [19]:
save_pickle(data_path, 'words_idx.pkl', vocab)

 file saved to: data/doc_level-sentiment/doc_level/words_idx.pkl


In [20]:
save_pickle(data_path, 'idx_words.pkl', idx_words)

 file saved to: data/doc_level-sentiment/doc_level/idx_words.pkl


In [21]:
save_pickle(data_path, 'data.pkl', data_list)

 file saved to: data/doc_level-sentiment/doc_level/data.pkl


In [22]:
save_pickle(data_path, 'label.pkl', label_list)

 file saved to: data/doc_level-sentiment/doc_level/label.pkl
